In [1]:
import pandas as pd
import numpy as np
from elements import elements
import elements.elements as elements
from astropy.table import Table

In [2]:
vdat = pd.read_csv('VALD_raw1', header=None, skiprows=2, nrows=400000, skip_blank_lines=False, dtype=str)
vdat = vdat.drop(13,axis=1)
vdatc = vdat.copy()
vdat = vdat[vdat[12].notna()]
#vdat = vdat[0:100000]
vdat = vdat.reset_index(drop=True)
#pd.options.display.max_rows = len(vdat[0])

In [3]:
el_list = pd.read_csv('elements.txt',delimiter=' ',header=None)
numbers = range(len(el_list[0])+1)
el_dict = dict(zip(el_list[0], numbers[1:]))

el = vdat[0].str.split(" ", n = 1, expand = True)
el[0] = el[0].replace({'\'':''}, regex=True)

for i in range(len(el[0])):
    e = el[0][i]
    if e in el_dict:
        el[0][i] = el_dict[e]
    else:
        el[0][i] = 0

el[1] = (el[1].replace({'\'':''}, regex=True).astype(int) - 1).apply(lambda x: '{0:0>2}'.format(x))
elm = el[0].map(str) + '.' + el[1].map(str)

In [4]:
labels = vdatc.copy() 
labels = labels[~labels[12].notna()][0:300000]
labels = pd.Series(labels.fillna('').values.tolist()).str.join(',')
labels = labels.replace({',,+':''}, regex=True)
labels = labels.replace({'\'  ':''}, regex=True)
labels = labels.replace({'\'':''}, regex=True)
labels = labels.replace({'  +':' '}, regex=True)

label_l = labels.iloc[::3]
label_l = label_l.reset_index(drop=True)
label_u = labels.iloc[1::3]
label_u = label_u.reset_index(drop=True)

In [22]:
wave = vdat[1].astype(float)
log_gf = (vdat[2].astype(float))
E_l = vdat[3].astype(float)
j_l = vdat[4].astype(float).apply(lambda x: '{0:0>2}'.format(x))
#label_l = pd.DataFrame(['2D' for s in range(len(vdat))])
E_u = vdat[5].astype(float)
j_u = vdat[6].astype(float).apply(lambda x: '{0:0>2}'.format(x))
#label_u = pd.DataFrame(['2D' for s in range(len(vdat))])
lande_l = vdat[7].astype(float)*1000
#lande_l = lande_l.replace(99000.0,0)
lande_u = vdat[8].astype(float)*1000
#lande_u = lande_u.replace(99000.0,0)
rad = vdat[10].astype(float)
stark = vdat[11].astype(float)
waals = vdat[12].astype(float)
zeroes = pd.DataFrame(np.zeros((len(vdat), 1)))

df_vald = pd.concat([wave, log_gf, elm, E_l, j_l, label_l, E_u, j_u, label_u, rad, stark, waals, zeroes, zeroes, zeroes, zeroes, zeroes, zeroes, zeroes, zeroes, lande_l, lande_u], axis=1)
df_vald.columns = range(df_vald.shape[1])
df_vald

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,300.000005,-8.115,0.00,11459.362,148.0,"Hb 8s2.9s1.3p4.1d1 X,3,2,+,148,0",44792.696,149.0,"Hb 8s2.3p4.4p1.1d1 B,3,1,-,148,13",7.437,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99000.0,99000.0
1,300.000045,-7.550,0.00,12112.801,150.0,"Hb 8s2.9s1.3p4.1d1 X,3,2,-,151,0",45446.130,150.0,"Hb 8s2.3p4.4p1.1d1 B,3,1,+,150,13",7.437,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99000.0,99000.0
2,300.000145,-7.296,0.00,11118.977,125.0,"Hb 8s2.9s1.3p4.1d1 X,3,2,+,124,3",44452.296,125.0,"Hb 8s2.3p4.4p1.1d1 B,3,1,-,126,15",7.419,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99000.0,99000.0
3,300.000285,-5.942,0.00,21760.061,196.0,"Hb 8s2.9s1.3p4.1d1 X,3,2,-,197,2",55093.364,196.0,"Hb 8s2.3p4.4p1.1d1 B,3,1,+,196,15",7.372,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99000.0,99000.0
4,300.000405,-6.272,0.00,14763.359,168.0,"Hb 8s2.9s1.3p4.1d1 X,3,2,+,168,0",48096.648,168.0,"Hb 8s2.3p4.4p1.1d1 B,3,1,-,167,13",7.441,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99000.0,99000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,307.420229,-6.299,0.00,19997.837,186.0,"Hb 8s2.9s1.3p4.1d1 X,3,2,+,187,2",52526.601,186.0,"Hb 8s2.3p4.4p1.1d1 B,3,1,-,186,14",7.482,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99000.0,99000.0
99996,307.420399,-7.809,0.00,12430.673,128.0,"Hb 8s2.9s1.3p4.1d1 X,3,2,+,127,4",44959.419,128.0,"Hb 8s2.3p4.4p1.1d1 B,3,1,-,129,15",7.421,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99000.0,99000.0
99997,307.420435,-6.645,0.00,10078.615,8.5,"Hb 3s2.4s2.1p4.5s1 X,2,0,e,8,5",42607.357,8.5,"Hb 3s2.4s2.1p3.5s2 A,2,1,f,9,22",7.180,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99000.0,99000.0
99998,307.420639,-9.577,0.00,2416.232,66.0,"Hb 8s2.9s1.3p4.1d1 X,3,2,-,66,0",34944.952,65.0,"Hb 8s2.3p4.4p1.1d1 B,3,1,+,64,15",7.331,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99000.0,99000.0


In [23]:
df_vald = df_vald[~(df_vald[2].astype(float)<1)]
df_vald.to_csv('VALD_linelist1.dat', header=None, index=None)

In [25]:
df_vald[df_vald[20]==99000]

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
263,300.020350,-1.570,11.01,298165.440,1.0,LS 2s2.2p5.3p 3D,331496.510,2.0,JK 2s2.2p5.(2P*<3/2>).4s 2[3/2]*,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99000.0,99000.0
430,300.032253,-2.350,10.01,305364.030,4.5,LS 2s2.2p4.(1D).3d 2G,338693.780,3.5,JK 2s2.2p4.(1D<2>).5f 2[4]*,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99000.0,99000.0
440,300.033153,-0.710,10.01,305364.030,4.5,LS 2s2.2p4.(1D).3d 2G,338693.680,4.5,JK 2s2.2p4.(1D<2>).5f 2[4]*,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99000.0,99000.0
479,300.036254,-3.370,8.01,232526.700,1.5,LS 2s2.2p2.(1D).3p 2P*,265856.000,2.5,LS 2s2.2p2.(3P).5d 2D,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99000.0,99000.0
555,300.042156,-0.810,10.01,305365.130,3.5,LS 2s2.2p4.(1D).3d 2G,338693.780,3.5,JK 2s2.2p4.(1D<2>).5f 2[4]*,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99000.0,99000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99796,307.407909,-1.708,25.00,17568.058,1.5,LS - a6D,50098.993,1.5,LS - v6P*,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99000.0,99000.0
99800,307.408100,-1.606,25.00,17567.997,1.5,LS - a6D,50098.912,1.5,LS - v6P*,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99000.0,99000.0
99802,307.408157,-1.738,25.00,17567.949,1.5,LS - a6D,50098.858,1.5,LS - v6P*,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99000.0,99000.0
99966,307.417998,0.480,16.05,596861.300,2.5,LS 2p6.6d 2D,629390.300,3.5,LS 2p6.7f 2F*,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99000.0,99000.0


In [7]:
# pd.options.display.max_columns = df_vald.shape[1]
# pd.set_option('display.expand_frame_repr', False)

with open('VALD_linelist1.dat', 'w') as f: 
    df_vald.to_string(f, col_space=10, index=False, header=None)